In [35]:
pip install atproto

In [36]:
# From the transformers package, import ViTFeatureExtractor and ViTForImageClassification
from transformers import ViTFeatureExtractor, ViTForImageClassification

# From the PIL package, import Image and Markdown
from PIL import Image

# import requests
import requests

# import torch
import torch

# import matplotlib
import matplotlib.pyplot as plt

# url getter for mpl
import urllib

import numpy as np

# import bluesky api
from atproto import Client

# import colab secrets to store login credentials
from google.colab import userdata

# datetime is necessary for caturday check
import datetime
import zoneinfo

In [37]:
# Load the feature extractor for the vision transformer
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
# Load the pre-trained weights from vision transformer
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [38]:
# 281: 'tabby, tabby cat'
# 282: 'tiger cat', 283: 'Persian cat', 284: 'Siamese cat, Siamese', 285: 'Egyptian cat', 286: 'cougar, puma, catamount, mountain lion, painter, panther, Felis concolor', 287: 'lynx, catamount', 288: 'leopard, Panthera pardus', 289: 'snow leopard, ounce, Panthera uncia', 290: 'jaguar, panther, Panthera onca, Felis onca', 291: 'lion, king of beasts, Panthera leo', 292: 'tiger, Panthera tigris', 293: 'cheetah, chetah, Acinonyx jubatus',
# 281 to 293
cat_labels = set()
for i in range(281, 294):
  cat_labels.add(i)

# these labels are to remove drawings, memes/reposts, and images with a lot of text respectively
bad_labels = {
917 : 'comic book', 916 : 'web site, website, internet site, site', 921 : 'book jacket, dust cover, dust jacket, dust wrapper'}

def test_bsky_image(url):

  f = urllib.request.urlopen(url)
  image = plt.imread(f, format='jpeg')
  # plt.imshow(image)
  inputs = feature_extractor(images=image, return_tensor="pt")
  pixel_values = inputs["pixel_values"]
  pixel_values = np.array(pixel_values)
  pixel_values = torch.tensor(pixel_values)
  outputs = model(pixel_values)
  logits = outputs.logits
  predicted_class_idx = logits.argmax(-1).item()
  sorted_preds = torch.argsort(logits, descending=True)[0]
  top_predictions = [sorted_preds[i].item() for i in range(50)] # 50 is semi-arbitrary based on our findings from testing pics
  top_values = [logits[0][pred].item() for pred in top_predictions]
  # print('label predictions', top_predictions)
  # print('values of predictions', top_values)
  found_cat_label = -1
  found_bad_label = -1
  bad_labels_found = []
  cat_score = 0
  for i, pred in enumerate(top_predictions):
    predicted_class = model.config.id2label[pred]
    # print(predicted_class)
    if pred in cat_labels:
      if found_cat_label == -1:
        found_cat_label = i
      cat_score += top_values[i]
    if pred in bad_labels:
      if found_bad_label == -1:
        found_bad_label = i
      bad_labels_found.append(pred)
      bad_labels_found.append(bad_labels[pred])
      cat_score -= top_values[i]
  # print(' ')
  print('    found cat label:', found_cat_label)
  print('    found bad label:', found_bad_label, bad_labels_found)
  would_pass = found_cat_label >= 0 and found_bad_label < 0
  # print('AI cat score: ', cat_score)
  print('    passed cat test', would_pass)
  return would_pass

In [43]:
# on colab you can set these userdata properties by clicking the key on the left bar, creating a secret, and giving the colab "notebook access"
BSKY_USERNAME = userdata.get('bsky_username')
BSKY_PASSWORD = userdata.get('bsky_password')

client = Client()
client.login(BSKY_USERNAME, BSKY_PASSWORD)

ProfileViewDetailed(did='did:plc:ktkc7jfakxzjpooj52ffc6ra', handle='tyrowo.bsky.social', associated=ProfileAssociated(chat=None, feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:ktkc7jfakxzjpooj52ffc6ra/bafkreiepqgg5tlozvv4bw5ficwwixflfprionhnli26hngc3hjn6ygpamu@jpeg', banner='https://cdn.bsky.app/img/banner/plain/did:plc:ktkc7jfakxzjpooj52ffc6ra/bafkreierasgybimbz3pfobqfmay5ufxm7jlqnjeiumdx7dyof6mcss7a4m@jpeg', created_at='2023-08-28T14:18:41.780Z', description='Self-taught professional software developer, exF2P MTG Arena Challenger, ex wr holder speedrunning HotD2100%, ex streamer, ex melee player. \nGrilling, coding, magic, and posting pictures of my cat. \nNO POLITICS\nhe/him', display_name='Tyro, ft Ricky', followers_count=565, follows_count=1374, indexed_at='2025-01-19T01:42:29.013Z', joined_via_starter_pack=None, labels=[], pinned_post=Main(cid='bafyreihszjgxkh5fmu5je5eldv

In [44]:
def like_post_and_add_user(post):
  user_did = post.author.did
  followed_user = client.follow(user_did).uri
  post_cid = post.cid
  post_uri = post.uri
  liked_post = client.like(uri=post_uri, cid=post_cid).uri
  print(f'      ✓✓✓ Successfully liked post and followed author.')
  # TODO - add this friend and the followed-user uri to the database

In [69]:
# input variables - x is target follows y is num of posts we want to look through, whichever end criteria we reach first
EMBEDDED_PIC = 'app.bsky.embed.images#view'
FEED_CATURDAY = 'at://did:plc:pmyqirafcp3jqdhrl7crpq7t/app.bsky.feed.generator/aaad4sb7tyvjw'
FEED_SIAMESE = 'at://did:plc:jv3qdc5vxujp6taaa7nte35i/app.bsky.feed.generator/aaac6wmikqyhq'
FEED_CATPICS = 'at://did:plc:q6gjnaw2blty4crticxkmujt/app.bsky.feed.generator/cv:cat'
FEED_CATS = 'at://did:plc:jfhpnnst6flqway4eaeqzj2a/app.bsky.feed.generator/cats'
URL_BEGIN = 'https://bsky.app/profile/'
URL_POST = '/post/'

def createPostUrl(feed_post):
  url_handle = feed_post.post.author.handle
  url_ending_index = feed_post.post.uri.find('.feed.post/') + 11
  url_ending = feed_post.post.uri[url_ending_index : ]
  return URL_BEGIN + url_handle + URL_POST + url_ending

# tyrowo.bsky.social/post/3lfzygdtqis2y

def follow_more_users(post_count, follows_count, feed):
  data = client.app.bsky.feed.get_feed({
      'feed': feed,
      'limit': post_count,
  }, headers={})

  feed = data.feed
  next_page = data.cursor

  count = 0

  for i, f in enumerate(data.feed):
    you_follow_them = f.post.author.viewer.following
    you_are_followed_by = f.post.author.viewer.followed_by
    if not f.post.embed or f.post.embed.py_type != EMBEDDED_PIC:
      print(f'{i} ✗ no pic for post {i}')
    elif you_follow_them or you_are_followed_by:
      print(f'{i} ✗ {f.post.author.handle + " follows you" if you_are_followed_by else ""}{" and " if you_follow_them and you_are_followed_by else ""}{"you already follow " + f.post.author.handle if you_follow_them else ""}.')
    ## TODO - add elif check here for if they are in the database as someone who was once our friend? not necessary, could just immediately cycle them back into the friend rotation, but do this if you don't want to readd people
    else:
      print(i, '✓', f.post.embed.images[0].fullsize)
      print(f'    post: {createPostUrl(f)}')
      is_cat = test_bsky_image(f.post.embed.images[0].fullsize)
      if is_cat:
        print(f'    ✓✓ successfully found cat pic at post {i}.')
        like_post_and_add_user(f.post)
        count += 1
        if count == follows_count:
          print(f'Successfully followed {count} new users!')
          return True
      else:
        print(f'    ✓✗ post {i} was not a cat pic')

  print(f'ran out of posts to check for new followers. followed {count} new users.')
  return False

  # print(data.feed[0].post.embed.images[0].fullsize)

In [70]:
CATURDAY_DOW = 'Saturday'
USER_TIMEZONE = "US/Eastern" # you should fill this in with your own timezone here
cur_timestamp = datetime.datetime.now(zoneinfo.ZoneInfo(USER_TIMEZONE))

dow = cur_timestamp.strftime("%A")
is_caturday = dow == CATURDAY_DOW

if is_caturday:
  print("IT'S CATURDAY! Checking the Caturday feed for new followers.")
  follow_more_users(100, 100, FEED_CATURDAY) # wanted more but the limit is actually 100
else:
  print("Sorry, today's not Caturday.")

IT'S CATURDAY! Checking the Caturday feed for new followers.
0 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:e4zgu3r5zxhbghdeqagdq2i2/bafkreicedkj6i4ac26fnr7ybzp3dbiezcxy4xbm4z6a3lkno2aput4fu5u@jpeg
    post: https://bsky.app/profile/thenexus.bsky.social/post/3lg2q2c5yu324


/usr/local/lib/python3.11/dist-packages/transformers/image_processing_utils.py:41: UserWarning: The following named arguments are not valid for `ViTFeatureExtractor.preprocess` and were ignored: 'return_tensor'
  return self.preprocess(images, **kwargs)


    found cat label: -1
    found bad label: 11 [917, 'comic book']
    passed cat test False
    ✓✗ post 0 was not a cat pic
1 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:52q33sp7jrvh3ruafwacxvev/bafkreiajhxai4oqsij5i6boyipytxme5g473ugon37lnqc7wmc57jwe524@jpeg
    post: https://bsky.app/profile/thewayofjc.bsky.social/post/3lg2pzlnx6k2a
    found cat label: 0
    found bad label: -1 []
    passed cat test True
    ✓✓ successfully found cat pic at post 1.
      ✓✓✓ Successfully liked post and followed author.
2 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:iw6bnf365diccdvymdexad5q/bafkreihxt5fe5gecxe5dz5hiie4asxrfzrxk4c6dfut3bjunggycmtnmfi@jpeg
    post: https://bsky.app/profile/jeryzner.bsky.social/post/3lg2pyaig7225
    found cat label: 28
    found bad label: -1 []
    passed cat test True
    ✓✓ successfully found cat pic at post 2.
      ✓✓✓ Successfully liked post and followed author.
3 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:jfgmiuhslk5a7d53inzw

In [71]:
follow_more_users(100, 100, FEED_SIAMESE)

0 ✗ tabitha-salamander.bsky.social follows you and you already follow tabitha-salamander.bsky.social.
1 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:qmujwnd543krzlr44bymnkbz/bafkreif54m5qltxhei5u2mqs6nt4srdkyywywdrxn5zcmwpokgupodmdbe@jpeg
    post: https://bsky.app/profile/marstead.com/post/3lg2pka24ac2b
    found cat label: 0
    found bad label: -1 []
    passed cat test True
    ✓✓ successfully found cat pic at post 1.
      ✓✓✓ Successfully liked post and followed author.
2 ✗ no pic for post 2
3 ✗ mariewinton.bsky.social follows you and you already follow mariewinton.bsky.social.
4 ✗ you already follow slop.guru.
5 ✗ you already follow wendywoodle.bsky.social.
6 ✗ necrotelicom.bsky.social follows you and you already follow necrotelicom.bsky.social.
7 ✗ you already follow bafflecore.bsky.social.
8 ✗ no pic for post 8
9 ✗ no pic for post 9
10 ✗ you already follow purpleodyssey.bsky.social.
11 ✗ no pic for post 11
12 ✗ you already follow thehotelcat.bsky.social.
13 ✗ you alr

False

In [49]:
follow_more_users(100, 100, FEED_CATS)

0 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:uzrm35oeukszmbg22xnfx4w3/bafkreid6zpvuuts5qxyw3x3ewm6argu6geeihafknxvwgg5ruj6v6v3cqu@jpeg
    post: at://did:plc:uzrm35oeukszmbg22xnfx4w3/app.bsky.feed.post/3lg2pbz67l22r
    found cat label: -1
    found bad label: -1 []
    passed cat test False
    ✓✗ post 0 was not a cat pic
1 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:q32j5o56ophhmuvtwf74qhlm/bafkreigalcv2o7nfpxyrg7kyu4bifi5lqkacst77achs6kfvzlos4k3bcy@jpeg
    post: at://did:plc:q32j5o56ophhmuvtwf74qhlm/app.bsky.feed.post/3lg2pbsazh22i
    found cat label: -1
    found bad label: 0 [917, 'comic book', 921, 'book jacket, dust cover, dust jacket, dust wrapper']
    passed cat test False
    ✓✗ post 1 was not a cat pic
2 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:ivmn7ip7kjq7zawoyykr7pcx/bafkreibwzt4v6rtn3sxggb577ygrg6kykb73qqjbnowadnv3mo5y6wspla@jpeg
    post: at://did:plc:ivmn7ip7kjq7zawoyykr7pcx/app.bsky.feed.post/3lg2pbl27pc2m
    found cat label: 0


False

In [50]:
follow_more_users(100, 100, FEED_CATPICS)

0 ✗ you already follow bodegacats.bsky.social.
1 ✗ you already follow catsofyore.bsky.social.
2 ✗ you already follow radiofreetom.bsky.social.
3 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:vmhrtckqi7ghdoqhf3lgvm4y/bafkreiaijx3n2v7qujtgv22eh2ax7wc3nfmczolwoybyqujxysk7b4e4ia@jpeg
    post: at://did:plc:vmhrtckqi7ghdoqhf3lgvm4y/app.bsky.feed.post/3lg23sbd7os2u
    found cat label: 0
    found bad label: 34 [916, 'web site, website, internet site, site']
    passed cat test False
    ✓✗ post 3 was not a cat pic
4 ✗ you already follow bebeneuwirth.bsky.social.
5 ✗ you already follow abbyhiggs.bsky.social.
6 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:eohxpvnt7zuqg4cfo5if7c4s/bafkreid7acbraoz7surtlucgmbcydqrcq7pcmuewq5bn2axgpbtoy4asf4@jpeg
    post: at://did:plc:eohxpvnt7zuqg4cfo5if7c4s/app.bsky.feed.post/3lg2d3hxou22z
    found cat label: -1
    found bad label: 4 [917, 'comic book', 921, 'book jacket, dust cover, dust jacket, dust wrapper']
    passed cat test False
 

False